<font size = "3">

**(Q1)** Make the following changes to the `Fraction` class from Lecture 1.

1. Modify the `__init__` method so that if a user creates a `Fraction` instance with a negative denominator, it returns an equivalent fraction with a positive denominator.

2. Remove the code that uses `gcd` to reduce a `Fraction` to lowest terms from the `__add__` method and add it to the `__init__` method instead. That is, anytime a `Fraction` instance is created, it is automatically put in reduced form.

3. Implement the following remaining simple arithmetic operators to the `Fraction` class: `__sub__`, `__mul__`, and `__truediv__`.

4. Implement the relational operators ``__gt__``, ``__ge__``, ``__lt__``, ``__le__``, and ``__ne__``.


In [1]:
def gcd(m, n):
    while m % n != 0:
        m, n = n, m % n
    return n


class Fraction:

    def __init__(self, num, den):
        if den == 0:
            raise ValueError("Denominator can't be zero")

        # Flip signs so denominator is always positive
        if den < 0:
            num = -num
            den = -den

        # Reduce to lowest terms in __init__ using gcd
        common = gcd(abs(num), den)
        self.num = num // common
        self.den = den // common

    def __str__(self):
        return f"{self.num}/{self.den}"

    def __repr__(self):
        return f"Fraction(num='{self.num}', den='{self.den}')"

    # Arithmetic operators
    def __add__(self, other):
        new_num = self.num * other.den + self.den * other.num
        new_den = self.den * other.den
        return Fraction(new_num, new_den)  

    def __sub__(self, other):
        new_num = self.num * other.den - self.den * other.num
        new_den = self.den * other.den
        return Fraction(new_num, new_den)

    def __mul__(self, other):
        new_num = self.num * other.num
        new_den = self.den * other.den
        return Fraction(new_num, new_den)

    def __truediv__(self, other):
        new_num = self.num * other.den
        new_den = self.den * other.num
        return Fraction(new_num, new_den)

    # Relational operators
    def __eq__(self, other):
        return self.num * other.den == other.num * self.den

    def __ne__(self, other):
        return not self.__eq__(other)

    def __lt__(self, other):
        return self.num * other.den < other.num * self.den

    def __le__(self, other):
        return self.num * other.den <= other.num * self.den

    def __gt__(self, other):
        return self.num * other.den > other.num * self.den

    def __ge__(self, other):
        return self.num * other.den >= other.num * self.den


In [ ]:
print(Fraction(1, 4) + Fraction(1, 2)) 
print(Fraction(3, -5))         
print(Fraction(1, 2) > Fraction(1, 3))  
print(Fraction(6, 8))                     

3/4
-3/5
True
3/4


<font size = "3">

**(Q2)** The circuit simulation from lecture 1 works in a backward direction. In other words, given a circuit, the ouptut is produced by working back through the input values, which in turn cause other outputs to be queried. The continues until the external input lines are found, at which point the user is asked for values. Modify the implementation so that the action is in the forward direction; upon receiving inputs the circuit produces an output.

**Hint**: You may develop another way to solve this problem, but I will outline a couple changes I made to develop my solution.

1. I changed the `LogicGate` class to have three attributes: `self.label`, `self.output`, and `self.pin_out`.

2. I changed the `LogicGate` class to have two methods: `send_output(self)` and `set_pin_out(self, target)`

3. I added another attribute to the `Connector` class: `self.value` which holds the output coming out of the "from gate".

4. I wrote a new `Circuit` class that contains a sequence of gates making up the circuit. 

5. After making all of my changes, I executed the circuit with the code below

In [3]:
class LogicGate:
    def __init__(self, lbl):
        self.label = lbl
        self.output = None
        self.pin_out = None 

    def send_output(self):
        if self.pin_out is not None:
            self.pin_out.value = self.output

    def set_pin_out(self, target): 
        self.pin_out = target


class BinaryGate(LogicGate):
    def __init__(self, lbl):
        super().__init__(lbl)
        self.pin_a = None
        self.pin_b = None

    def get_input_a(self):
        if self.pin_a is None:
            return int(input(f"Enter pin A input for gate {self.label}: "))
        else:
            return self.pin_a.value  

    def get_input_b(self):
        if self.pin_b is None:
            return int(input(f"Enter pin B input for gate {self.label}: "))
        else:
            return self.pin_b.value  

    def set_pin(self, source):
        if self.pin_a is None:
            self.pin_a = source
        elif self.pin_b is None:
            self.pin_b = source
        else:
            raise RuntimeError("Error: no available pins")


class UnaryGate(LogicGate):
    def __init__(self, lbl):
        super().__init__(lbl)
        self.pin = None

    def get_input(self):
        if self.pin is None:
            return int(input(f"Enter pin input for gate {self.label}: "))
        else:
            return self.pin.value  

    def set_pin(self, source):
        if self.pin is None:
            self.pin = source
        else:
            raise RuntimeError("Error: PIN IS NOT AVAILABLE")


class AndGate(BinaryGate):
    def __init__(self, lbl):
        super().__init__(lbl)

    def perform_gate_logic(self):
        a = self.get_input_a()
        b = self.get_input_b()
        self.output = a * b


class OrGate(BinaryGate):
    def __init__(self, lbl):
        super().__init__(lbl)

    def perform_gate_logic(self):
        a = self.get_input_a()
        b = self.get_input_b()
        self.output = max(a, b)


class NotGate(UnaryGate):
    def __init__(self, lbl):
        super().__init__(lbl)

    def perform_gate_logic(self):
        x = self.get_input()
        self.output = (x + 1) % 2


class Connector:
    def __init__(self, from_gate, to_gate):
        self.from_gate = from_gate
        self.to_gate = to_gate
        self.value = None 

        from_gate.set_pin_out(self)
        to_gate.set_pin(self)      


class Circuit:
    def __init__(self, sequence):
        self.sequence = sequence  

    def run_circuit(self):
        for level in self.sequence:
            for gate in level:
                gate.perform_gate_logic()  
                gate.send_output()      

        final_gate = self.sequence[-1][-1]
        print(f"Circuit output: {final_gate.output}")


In [4]:
g1 = AndGate("G1")
g2 = AndGate("G2")
g3 = OrGate("G3")
g4 = NotGate("G4")

c1 = Connector(g1, g3)
c2 = Connector(g2, g3)
c3 = Connector(g3, g4)

level_1 = [g1, g2]
level_2 = [g3]
level_3 = [g4]
sequence = [level_1, level_2, level_3]

circuit = Circuit(sequence)

circuit.run_circuit()

Circuit output: 1


<font size = "3">

**(Q3)** Consider the problem of solving an $n\times n$ lower-triangular system of equations $L\mathbf{x} = \mathbf{b}$, where $\mathbf{x}$ is unknown.

$$
\underbrace{\begin{bmatrix}
\ell_{11} & 0         & \cdots & 0 \\
\ell_{21} & \ell_{22} & \ddots & \vdots \\
\vdots    & \vdots    & \ddots & 0 \\
\ell_{n1} & \ell_{n2} & \cdots & \ell_{nn}
\end{bmatrix}}_{L}
\underbrace{\begin{bmatrix}
x_1 \\ x_2 \\ \vdots \\ x_n
\end{bmatrix}}_{\mathbf{x}}
=
\underbrace{\begin{bmatrix}
b_1 \\ b_2 \\ \vdots \\ b_n
\end{bmatrix}}_{\mathbf{b}}.
$$

We assume $\ell_{ii} \neq 0$ for every $i = 1, 2, \dots, n$ so that a unique solution exists.

The *forward-substitution* algorithm can be used to compute the solution as follows:

$$\begin{align*}
x_1 &= \frac{b_1}{\ell_{11}}\\[2pt]
x_2 &= \frac{b_2 - \ell_{21}x_1}{\ell_{22}}\\
x_3 &= \frac{b_3 - \ell_{31}x_1 - \ell_{32}x_2}{\ell_{33}}\\
&\vdots\\
x_i &= \frac{b_i - \sum_{j=1}^{i-1}\ell_{ij}x_j}{\ell_{ii}}\\
&\vdots\\
x_n &= \frac{b_n - \sum_{j=1}^{n-1}\ell_{ij}x_j}{\ell_{nn}}
\end{align*}




1. Implement the forward substitution algorithm as a function `solve_lower_triangular(L, b)`

2. How many operations does your implementation require? "Operations" include assignnents, addition/subtraction, and multiplication/division. Briefly explain how you arrived at your answer.

3. What is the computational cost of the algorithm in terms of "Big-Oh" notation?

4. Verify your answer to part 3 empirically using the `timeit` library for different values of `n`

In [5]:
import numpy as np

def solve_lower_triangular(L, b):
    n = len(b)
    x = np.zeros(n)

    for i in range(n):
        # sum of L[i,j] * x[j] for j = 0, ..., i-1
        total = 0
        for j in range(i):
            total += L[i, j] * x[j]

        x[i] = (b[i] - total) / L[i, i]

    return x


Check to see if it worked:

In [10]:
# Here's a good way to create lower triangular systems with unique solutions
import numpy as np 

n = 50 

L = np.tril(np.random.randn(n,n))
L += np.diag(5*(np.random.rand(n,) + 1)) # ensure matrix is "sufficiently" non-singular

b = np.random.randn(n,)

# Here's how you can test if your function was implemented correctly

def solve_triangular(L, b):
    # will obviously need to change this
    return np.linalg.solve(L, b)

n = 50

L = np.tril(np.random.randn(n,n))
L += np.diag(5*(np.random.rand(n,) + 1)) # ensure matrix is "sufficiently" non-singular

x_true = np.ones(n,)
b = L @ x_true 

my_x = solve_triangular(L, b)

# true solution should be all ones... should get a really small number here
print(max(abs(my_x - 1)))


6.661338147750939e-16


In [11]:
from timeit import timeit
import matplotlib.pyplot as plt

n_vals = [50, 100, 200, 400, 800]
times = []
num_repeats = 10

for n in n_vals:
    L = np.tril(np.random.randn(n, n))
    L += np.diag(5 * (np.random.rand(n,) + 1))
    b = np.random.randn(n,)

    t = timeit(
        stmt="solve_lower_triangular(L, b)",
        number=num_repeats,
        globals={"solve_lower_triangular": solve_lower_triangular, "L": L, "b": b}
    )
    avg_time = t / num_repeats
    times.append(avg_time)
    print(f"n = {n:>4d}, avg time = {avg_time:.4e} seconds")

# Show that doubling n roughly quadruples the time (consistent with O(n^2))
print("\nRatios (should be ~4 if O(n^2)):")
for i in range(1, len(times)):
    ratio = times[i] / times[i - 1]
    print(f"n={n_vals[i]:>4d} / n={n_vals[i-1]:>4d}: ratio = {ratio:.2f}")


n =   50, avg time = 1.8682e-04 seconds
n =  100, avg time = 7.7222e-04 seconds
n =  200, avg time = 3.0263e-03 seconds
n =  400, avg time = 1.1679e-02 seconds
n =  800, avg time = 4.8326e-02 seconds

Ratios (should be ~4 if O(n^2)):
n= 100 / n=  50: ratio = 4.13
n= 200 / n= 100: ratio = 3.92
n= 400 / n= 200: ratio = 3.86
n= 800 / n= 400: ratio = 4.14


<font size = "4">

**(Q4)** Consider the function $f(n) = \frac{2}{3}n^2 + 500n + 50$. 

- Using the precise definition of $\mathcal{O}(\cdot)$, show that $f(n) = \mathcal{O}(n^2)$

- Using the precise definition of $\mathcal{\Theta}(\cdot)$, show that $f(n) = \mathcal{\Theta}(n^2)$

Part 1: Show that $f(n) = \mathcal{O}(n^2)$
Using the definition: We need to find a constant $C > 0$ and a positive integer $N$ such that

$$|f(n)| \leq C \cdot |n^2| \quad \text{for } n \geq N$$

For $n \geq 1$, we have $f(n) > 0$ and $n^2 > 0$, so we can drop the absolute values.

For $n \geq 1$:
$$f(n) = \frac{2}{3}n^2 + 500n + 50$$

Since $n \geq 1$ implies $n \leq n^2$ and $1 \leq n^2$:

$$f(n) = \frac{2}{3}n^2 + 500n + 50 \leq \frac{2}{3}n^2 + 500n^2 + 50n^2 = \frac{1552}{3}n^2$$

So choosing $C = \frac{1552}{3}$ and $N = 1$, we have:

$$f(n) \leq C \cdot n^2 \quad \text{for all } n \geq N$$

Therefore $f(n) = \mathcal{O}(n^2)$. $\blacksquare$

Part 2: Show that $f(n) = \Theta(n^2)$
Using the definition we need to show both:

$f(n) = \mathcal{O}(n^2)$ — there exists $C > 0$ and $N$ such that $f(n) \leq C \cdot n^2$ for $n \geq N$
$f(n) = \Omega(n^2)$ — there exists $c > 0$ and $M$ such that $c \cdot n^2 \leq f(n)$ for $n \geq M$
Upper bound $(\mathcal{O})$: Already shown above with $C = \frac{1552}{3}$, $N = 1$.

Lower bound $(\Omega)$: For $n \geq 1$, all terms are positive:

$$f(n) = \frac{2}{3}n^2 + 500n + 50 \geq \frac{2}{3}n^2$$

So choosing $c = \frac{2}{3}$ and $M = 1$, we have:

$$c \cdot n^2 \leq f(n) \quad \text{for all } n \geq M$$

Therefore $f(n) = \Omega(n^2)$.

Since $f(n) = \mathcal{O}(n^2)$ and $f(n) = \Omega(n^2)$, by definition:

$$f(n) = \Theta(n^2) \quad \blacksquare$$

<font size = "3">

**(Q5)** Implement a class `TransformHistory` that contains a Pandas `DataFrame` object along with an "undo_stack" and a "redo_stack". This class allows a user to apply a sequence of transformations to a DataFrame while providing a mechanism for undoing/redoing transformations. The outline of the class is below (using the `Stack` class from Lecture 4).

**Hint:** Think of the "Back" and "Forward" options in your browser, or the "undo" "redo" features when writing a word document.

In [16]:
class Stack:
    def __init__(self):
        self._items = []

    def is_empty(self):
        return not bool(self._items)

    def push(self, item):
        self._items.append(item)

    def pop(self):
        return self._items.pop()

    def peek(self):
        return self._items[-1]

    def size(self):
        return len(self._items)


class TransformHistory:
    def __init__(self, df):
        self.data = df.copy()
        self.undo_stack = Stack()
        self.redo_stack = Stack()

    def set_data(self, df):
        """Set the working DataFrame
        - overwrite self.data with a copy of df
        - reset both undo and redo stacks"""
        self.data = df.copy()
        self.undo_stack = Stack()
        self.redo_stack = Stack()

    def apply(self, fn, label=None): # label is optional, allows naming a transformation

        """Apply the transformation fn to the DataFrame
        - Clear the redo stack
        - Add a copy of the current data to the undo stack
        - Overwrite the current data by applying the
            transformation to a copy of the current data"""
        self.redo_stack = Stack()
        self.undo_stack.push(self.data.copy())
        self.data = fn(self.data.copy())

    def undo(self):
        """Undo the most recent transformation.
        If there is no transformation to undo, leave
        data unchanged"""
        if not self.undo_stack.is_empty():
            self.redo_stack.push(self.data.copy())
            self.data = self.undo_stack.pop()

    def redo(self):
        """Redo the most recent 'undone' transformation.
        If there is no transformation to redo, leave
        data unchanged"""
        if not self.redo_stack.is_empty():
            self.undo_stack.push(self.data.copy())
            self.data = self.redo_stack.pop()


<font size = "3">

See below for example usage of the `TransformHistory` class.

In [18]:
import pandas as pd

def add_y(d):
    return d.assign(y=d["x"] + 10)

def filter_ge_2(d):
    return d.query("x >= 2")


# initialize with empty DataFrame
h = TransformHistory(pd.DataFrame())

# replace with simple 1-column data frame
h.set_data(pd.DataFrame({"x": [1, 2, 3]}))

h.apply(add_y)
print(h.data, "\n")
#     x   y
# 0   1  11
# 1   2  12
# 2   3  13

h.apply(filter_ge_2, "filter")
print(h.data, "\n")
#     x   y
# 1   2  12
# 2   3  13

h.undo()
print(h.data, "\n")
#     x   y
# 0   1  11
# 1   2  12
# 2   3  13

h.redo()
print(h.data, "\n")
#     x   y
# 1   2  12
# 2   3  13

   x   y
0  1  11
1  2  12
2  3  13 

   x   y
1  2  12
2  3  13 

   x   y
0  1  11
1  2  12
2  3  13 

   x   y
1  2  12
2  3  13 

